### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----
#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

## Input Processing

* Read data from file "purchase_data.csv"
* Sample the 'head' and tail of the data set

In [238]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [239]:
# Show the 'head' rows of the data set
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [240]:
# Show the 'tail' rows of the data set
purchase_data.tail()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,101,Final Critic,4.19
779,779,Ennrian78,24,Male,50,Dawn,4.60


## Player and Item Count
* Display the total number of players and unique items


In [281]:
# Print total number of players and items
playerCnt = purchase_data["SN"].nunique()
itemCnt = purchase_data["Item ID"].nunique()
print(f'The total number of players in the data set is : {playerCnt}')
print(f'The total number of unique items in the data set is : {itemCnt}')

The total number of players in the data set is : 576
The total number of unique items in the data set is : 183


## Notes / Experiments (delete for final)


In [242]:
tmpdf = purchase_data[['Item ID', 'Item Name', 'Price']].drop_duplicates()
#tmpdf.nunique()
dupList = list(tmpdf[tmpdf['Item Name'].duplicated()]['Item Name'])

In [243]:
a = pd.DataFrame()
for x in dupList :
    a = a.append(tmpdf.loc[tmpdf['Item Name'] == x, ['Item ID', 'Item Name', 'Price']])
a

,Item ID,Item Name,Price
2,92,Final Critic,4.88
99,101,Final Critic,4.19
121,1,Crucifer,3.26
251,23,Crucifer,1.99
24,141,Persuasion,3.19
608,132,Persuasion,3.33
166,30,Stormcaller,2.21
751,180,Stormcaller,3.36


## Purchasing Analysis (counts and basic statistics)
* Run basic calculations to obtain number of unique items, average price, etc.

* Create a summary data frame to hold the results

* Optional: give the displayed data cleaner formatting

* Display the summary data frame


In [244]:
# Capture count of unique values per column for comparison to row count
cntUnique = purchase_data.aggregate({'Purchase ID':'nunique', 'SN':'nunique', 'Item ID':'nunique', 'Item Name':'nunique', 'Age':'nunique', 'Price':'nunique'})

# Capture maximum, minimum and mean values for 'Price' and 'Age'
maxValues = purchase_data.aggregate({'Price':'max', 'Age':'max'})
minValues = purchase_data.aggregate({'Price':'min', 'Age':'min'})
avgValues = purchase_data.aggregate({'Price':'mean', 'Age':'mean'})

# Create the summary dataframe, clear NaN values, round 'Average' values to 2 decimal points
summaryDF = pd.DataFrame({'Counts': cntUnique, 'Maximum': maxValues, 'Minimum': minValues, 'Average': avgValues})
summaryDF['Average'] = summaryDF['Average'].round(decimals=2)
summaryDF.fillna('',inplace=True)

# Show the summary dataframe
summaryDF

,Counts,Maximum,Minimum,Average
Age,39,45,7,22.71
Item ID,183,,,
Item Name,179,,,
Price,145,4.99,1,3.05
Purchase ID,780,,,
SN,576,,,


## Gender Demographics

* Percentage and Count of Male Players

* Percentage and Count of Female Players

* Percentage and Count of Other / Non-Disclosed

* Helpful links :

    * Unique and duplicates : https://stackoverflow.com/questions/38309729/count-unique-values-with-pandas-per-groups

    * Styles : https://pandas.pydata.org/pandas-docs/stable/style.html#Finer-Control:-Display-Values


In [282]:
# Create summary dataframe of Player ('SN') counts grouped by 'Gender' column values. 
genderSummary  = purchase_data.groupby('Gender')[['SN']].nunique()
genderSummary.rename(columns={'SN':'Gender Count'}, inplace=True)

# Add column to summary dataframe for Player ('SN') percentage by 'Gender'
genderSummary['Player Percentage'] = (genderSummary['Gender Count'] / playerCnt)*100

# Show formatted summary dataframe
genderSummary.style.format({'Player Percentage':"{:.2f}%"})

,Gender Count,Player Percentage
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [4]:
# Sample output

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender

* Create a summary data frame to hold the results

* Optional: give the displayed data cleaner formatting

* Display the summary data frame

In [283]:
# Create summary dataframe of Purchase Counts and Purchase Totals by Gender
genderPurchaseSum = purchase_data.groupby('Gender').aggregate({'Purchase ID':'count', 'Price':'sum'})
genderPurchaseSum.rename(columns={'Purchase ID':'Purchase Count', 'Price':'Total Purchase Value'}, inplace=True)

# Add column to summary dataframe for Average Purchase Value by 'Gender'
genderPurchaseSum['Average Purchase Price'] = genderPurchaseSum['Total Purchase Value'] / genderPurchaseSum['Purchase Count']

# Add column to summary dataframe for Average Purchase Value per Person by 'Gender'
# using gender player counts from genderSummary dataframe (from previous cell)
genderPurchaseSum['Avg Total Purchase per Person'] = genderPurchaseSum['Total Purchase Value'] / genderSummary['Gender Count']

# Show formatted summary dataframe
genderPurchaseSum.style.format({'Total Purchase Value':"${:.2f}", 'Average Purchase Price':"${:.2f}", 'Avg Total Purchase per Person':"${:.2f}"})

,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,$1967.64,$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


In [5]:
# Sample output

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics
* Establish bins for ages

* Categorize the existing players using the age bins. Hint: use pd.cut()

* Calculate the numbers and percentages by age group

* Create a summary data frame to hold the results

* Optional: round the percentage column to two decimal points

* Display Age Demographics Table


In [289]:
# Capture age for each player
players = (purchase_data[['SN', 'Age']].drop_duplicates())

# Define age interval bin list and age grouping label list
bins   = [0,9,14,19,24,29,34,39,players['Age'].max()]
groups = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# Create age demographic summary dataframe, grouped by age range
ageSummary = pd.DataFrame({'Age Group':pd.cut(players['Age'], bins, labels=groups), 'Total Count':players['Age']})
ageSummary = ageSummary.groupby('Age Group').count()

# Compute Percentage of Players per Age Group
ageSummary['Percentage of Players'] = (ageSummary['Total Count'] * 100) / playerCnt

# Formated demographic summary output
ageSummary.style.format({'Percentage of Players':"{:.2f}%"})

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [6]:
# Sample output

,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)
* Bin the purchase_data data frame by age

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below

* Create a summary data frame to hold the results

* Optional: give the displayed data cleaner formatting

* Display the summary data frame

In [306]:
# Create summary dataframe for purchases by age
agePurchaseSum = purchase_data[['Purchase ID', 'Price']].copy()
agePurchaseSum.rename(columns={'Purchase ID':'Purchase Count', 'Price':'Total Purchase Value'}, inplace=True)

# Add column for age groupings, then group using age groupings
agePurchaseSum['Age Group'] = pd.cut(purchase_data['Age'], bins, labels=groups)
agePurchaseSum = agePurchaseSum.groupby('Age Group').aggregate({'Purchase Count':'count', 'Total Purchase Value':'sum'})

# Add columns Average Purchase Value and Average Purchase Value per Person
agePurchaseSum['Average Purchase Value'] = agePurchaseSum['Total Purchase Value'] / agePurchaseSum['Purchase Count']
agePurchaseSum['Average Purchase Value per Person'] = agePurchaseSum['Total Purchase Value'] / ageSummary['Total Count']

# Formated purchase demographic summary output
agePurchaseSum.style.format({'Total Purchase Value':"${:.2f}", 'Average Purchase Value':"${:.2f}", 
                           'Average Purchase Value per Person':"${:.2f}"})

,Purchase Count,Total Purchase Value,Average Purchase Value,Average Purchase Value per Person
Age Group,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,$1114.06,$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
40+,13,$38.24,$2.94,$3.19


In [7]:
# Sample output

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19
<10,23,$3.35,$77.13,$4.54


## Top Spenders

* Run basic calculations to obtain the results in the table below

* Create a summary data frame to hold the results

* Sort the total purchase value column in descending order

* Optional: give the displayed data cleaner formatting

* Display a preview of the summary data frame


In [288]:
# Establish Player group object
spendGroup = purchase_data.groupby('SN')

# Initialize summary DataFrame with 'Purchase Count' and 'Average Purchase Amount' columns
spendSummary = spendGroup.aggregate({'Item ID':'count', 'Price':'mean'})
spendSummary.rename(columns={'Item ID':'Purchase Count', 'Price':'Average Purchase Amount'}, inplace=True)

# Add column for 'Total Purchase Value' and sort 'Total Purchase Value' in descending order
spendSummary['Total Purchase Value'] = spendGroup['Price'].sum()
spendSummary.sort_values(by='Total Purchase Value', ascending=False, inplace=True)

# Formatted output of summary [1st 5 rows]
spendSummary.iloc[0:5].style.format({'Average Purchase Amount':"${:.2f}", 'Total Purchase Value':"${:.2f}"})

,Purchase Count,Average Purchase Amount,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [8]:
# Sample output

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns

* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value

* Create a summary data frame to hold the results

* Sort the purchase count column in descending order

* Optional: give the displayed data cleaner formatting

* Display a preview of the summary data frame



In [80]:
# 'Item ID' identifies uniquely each purchase and associated item 'Price' 
#    hence 'Price' value by 'Item ID'/'Item Name' is the same for each purchase
#    i.e. 'Price' value = min, max, mean, first and last aggregates

# Establish group object for purchase_data['Item ID', 'Item Name']
itemGroup   = purchase_data.groupby(['Item ID', 'Item Name'])

# Initialize summary DataFrame with 'Purchase Count' and 'Item Price' columns
itemSummary = itemGroup.aggregate({'Item ID':'count', 'Price':'first'})
itemSummary.rename(columns={'Item ID':'Purchase Count','Price':'Item Price'}, inplace=True)

# Add column for 'Total Purchase Value' and sort 'Purchase Count' in descending order
itemSummary['Total Purchase Value'] = itemGroup['Price'].sum()
itemSummary.sort_values(by='Purchase Count', ascending=False, inplace=True)

# Formatted output of summary
itemSummary.iloc[0:5].style.format({'Item Price':"${:.2f}", 'Total Purchase Value':"${:.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [9]:
# Sample output

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order

* Optional: give the displayed data cleaner formatting

* Display a preview of the data frame


In [133]:
# Sort summary by 'Total Purchase Value' in descending order
itemSummary.sort_values(by='Total Purchase Value', ascending=False, inplace=True)

# Formatted output of summary
itemSummary.iloc[0:5].style.format({'Item Price':"${:.2f}", 'Total Purchase Value':"${:.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


In [10]:
# Sample output

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
